In [13]:
import requests
import time
import configparser
from urllib.parse import quote

# Wikipedia page to monitor
WIKIPEDIA_PAGE = "Opinion_polling_for_the_2024_United_Kingdom_general_election"
API_URL = f"https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles={quote(WIKIPEDIA_PAGE)}&rvlimit=1&format=json"

# Slack configuration
config = configparser.ConfigParser()
config.read('config.ini')
SLACK_WEBHOOK_URL = config['Slack']['SLACK_WEBHOOK_URL']

# Get the latest revision id of the wiki page
def get_latest_revision_id():
    response = requests.get(API_URL)
    data = response.json()
    pages = data['query']['pages']
    for page_id in pages:
        revision_id = pages[page_id]['revisions'][0]['revid']
        return revision_id
    return None

def send_slack_notification():
    message = {
        "text": f"The Wikipedia page '{WIKIPEDIA_PAGE.replace('_', ' ')}' has been updated. Check it out here: https://en.wikipedia.org/wiki/{WIKIPEDIA_PAGE}"
    }
    response = requests.post(SLACK_WEBHOOK_URL, json=message)
    if response.status_code == 200:
        print("Slack notification sent successfully")
    else:
        print(f"Failed to send Slack notification: {response.status_code}")

def monitor_wikipedia_page(interval=60):
    last_revision_id = get_latest_revision_id()
    while True:
        time.sleep(interval)
        current_revision_id = get_latest_revision_id()
        if current_revision_id != last_revision_id:
            print(f"Change detected! Revision ID: {current_revision_id}")
            send_slack_notification()
            last_revision_id = current_revision_id

if __name__ == "__main__":
    monitor_wikipedia_page(interval=60 * 60)  # Check every hour


KeyboardInterrupt: 